## Sentiment Analysis

In this exercise we use the IMDb-dataset, which we will use to perform a sentiment analysis. The code below assumes that the data is placed in the same folder as this notebook. We see that the reviews are loaded as a pandas dataframe, and print the beginning of the first few reviews.

In [3]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping # Added EarlyStopping
from tensorflow.keras.layers import Dropout
from tensorflow.keras import regularizers
from numpy.random import seed, randint

reviews = pd.read_csv('reviews.txt', header=None)
labels = pd.read_csv('labels.txt', header=None)
Y = (labels=='positive').astype(np.int_)

print(type(reviews))
print(reviews.head())

<class 'pandas.core.frame.DataFrame'>
                                                   0
0  bromwell high is a cartoon comedy . it ran at ...
1  story of a man who has unnatural feelings for ...
2  homelessness  or houselessness as george carli...
3  airport    starts as a brand new luxury    pla...
4  brilliant over  acting by lesley ann warren . ...


**(a)** Split the reviews and labels in test, train and validation sets. The train and validation sets will be used to train your model and tune hyperparameters, the test set will be saved for testing. Use the `CountVectorizer` from `sklearn.feature_extraction.text` to create a Bag-of-Words representation of the reviews. Only use the 10,000 most frequent words (use the `max_features`-parameter of `CountVectorizer`).

**(b)** Explore the representation of the reviews. How is a single word represented? How about a whole review?

**(c)** Train a neural network with a single hidden layer on the dataset, tuning the relevant hyperparameters to optimize accuracy. 

**(d)** Test your sentiment-classifier on the test set.

**(e)** Use the classifier to classify a few sentences you write yourselves. 

In [9]:
#A)

X_trainval, X_test, y_trainval, y_test = train_test_split(reviews[0], Y[0] , random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, 
                                                  random_state=43)

count_vectorizer = CountVectorizer(max_features=10000)
X_train_bow = count_vectorizer.fit_transform(X_train)
X_val_bow = count_vectorizer.transform(X_val)
X_test_bow = count_vectorizer.transform(X_test)

print(X_train_bow.shape)
print(X_val_bow.shape)
print(X_test_bow.shape)

(14062, 10000)
(4688, 10000)
(6250, 10000)


In [10]:
#B) 
# for a single word
print("Single words")
word_counts = np.asarray(X_train_bow.sum(axis=0)).flatten()
vocab = count_vectorizer.get_feature_names_out()

word_freq = list(zip(vocab, word_counts))
word_freq_sorted = sorted(word_freq, key=lambda x: x[1], reverse=True)
# Print top 50 most common words
for word, freq in word_freq_sorted[:50]:
    print(f"{word}: {freq}")

print("\n")

# for a review
print("Review words")
sample_idx = 0
sample_vector = X_train_bow[sample_idx]

vocab = count_vectorizer.get_feature_names_out()

word_indices = sample_vector.nonzero()[1]
word_counts = sample_vector.data

bow_representation = {vocab[i]: count for i, count in zip(word_indices, word_counts)}

for word, count in sorted(bow_representation.items(), key=lambda x: x[1], reverse=True):
    print(f"{word}: {count}")

Single words
the: 189639
and: 92662
of: 81947
to: 76525
is: 60238
br: 57951
it: 54143
in: 52645
this: 42800
that: 41490
was: 27177
as: 26199
with: 24930
for: 24876
movie: 24663
but: 24014
film: 22633
on: 19237
you: 19123
not: 17170
he: 16856
are: 16513
his: 16441
have: 15701
be: 15166
one: 15123
all: 13579
at: 13266
they: 12683
by: 12631
who: 12078
an: 12051
from: 11532
so: 11485
like: 11294
there: 10550
her: 10346
or: 10061
just: 10059
about: 9690
out: 9667
if: 9386
has: 9364
what: 9080
some: 8931
good: 8460
can: 8205
more: 8039
she: 8001
when: 7971


Review words
the: 4
it: 3
and: 2
wacky: 2
see: 2
this: 1
movie: 1
is: 1
not: 1
only: 1
funniest: 1
film: 1
ever: 1
created: 1
greatest: 1
my: 1
hats: 1
off: 1
to: 1
mr: 1
mrs: 1
rest: 1
of: 1
cast: 1
good: 1
morning: 1
satan: 1
want: 1
post: 1
go: 1
now: 1


## Single word representation
<b>Single words are represented in a way that, the higher is the word in Bag-of-Words, the more frequently it is used in a review. We can say they are represented in a hierarchy most to least frequently used words</b>

## Whole review representation
<b>In whole review representation we can see how often each word was used in the context of that, exact review. We can see how repetitive the person was while writing the review</b>


In [14]:
#C)
num_classes = 10
input_size = reviews[0].shape[0]
seed(0)
tf.random.set_seed(0)

model = Sequential() # initialize a neural network
model.add(Dense(units = 32, activation='tanh', input_dim=input_size)) # add a hidden layer
model.add(Dense(units = num_classes, activation='softmax')) #add the output layer

sgd = SGD(learning_rate = 0.1)
model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'])

history = model.fit(X_train_bow, y_train_oh, 
                    epochs=10, batch_size=50, verbose=1,
                    validation_data=(X_val_bow, y_val_oh))

NameError: name 'y_train_oh' is not defined